In [34]:
import pandas as pd
import backtrader as bt
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import datetime

dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H.%M.%S')
data = pd.read_csv("data.csv", parse_dates=['timestamp'], index_col='idx', date_parser=dateparse)
strategy_data = data.resample(rule='15T', on='timestamp').agg(
    {'open': 'first',
     'high': 'max',
     'low': 'min',
     'close': 'last',
     'volume': 'sum'
     })
strategy_data = strategy_data.reset_index()
strategy_data.head(5)

,timestamp,open,high,low,close,volume
0,2022-12-01 00:00:00,17165.44,17194.00,17133.35,17188.29,3008
1,2022-12-01 00:15:00,17188.85,17236.29,17159.61,17202.17,5286
2,2022-12-01 00:30:00,17202.17,17210.53,17151.02,17155.80,2505
3,2022-12-01 00:45:00,17155.80,17179.38,17122.65,17161.55,2565
4,2022-12-01 01:00:00,17161.55,17170.32,17134.07,17145.88,2172


In [55]:
from Bbands import StFetcher
from backtrader.feeds import PandasData

bt_data = PandasData(dataname=strategy_data,
                     datetime='timestamp',
                     high='high',
                     low='low',
                     open='open',
                     close='close',
                     volume='volume')
cerebro = bt.Cerebro()
broker = cerebro.broker
broker.setcash(100000.0)
cerebro.adddata(bt_data)
cerebro.optstrategy(StFetcher,period=[20, 50], bias=[1.1, 1.2])
cerebro.run()

2023-01-17 14:21:25.146 | INFO     | Bbands:__init__:13 - parameter is 20-1.1
2023-01-17 14:21:25.148 | INFO     | Bbands:__init__:13 - parameter is 20-1.2
2023-01-17 14:21:25.149 | INFO     | Bbands:__init__:13 - parameter is 50-1.1
2023-01-17 14:21:25.151 | INFO     | Bbands:__init__:13 - parameter is 50-1.2
2023-01-17 14:21:25.192 | INFO     | Bbands:next:31 - create close signal,0.0,lower is 17120.2168,close is 17114.68
2023-01-17 14:21:25.193 | INFO     | Bbands:next:31 - create close signal,0.0,lower is 17146.4335,close is 17142.23
2023-01-17 14:21:25.194 | INFO     | Bbands:next:31 - create close signal,0.0,lower is 17146.4335,close is 17142.23
2023-01-17 14:21:25.194 | INFO     | Bbands:next:31 - create close signal,1.0,lower is 17119.233999999997,close is 17139.15
2023-01-17 14:21:25.198 | INFO     | Bbands:next:45 - create close signal,1.0,lower is 17119.233999999997,close is 17139.15
2023-01-17 14:21:25.196 | INFO     | Bbands:next:31 - create close signal,1.0,lower is 17144

[[<backtrader.cerebro.OptReturn at 0x7f8f0758c220>],